### SQL Challenge

##### 1. Build a query to count the number of loans per customer

SELECT custid, count(loanid)
FROM all_loans
GROUP BY custid
ORDER BY count(loanid) desc, custid;

##### 2. Write a query to identify if a customer had more than one loan at the same time

SELECT a.custid
FROM all_loans a
, all_loans b
where a.custid=b.custid
and b.loanid>a.loanid
and (b.approvedate <a.payoffdate or a.approvedate < a.writeoffdate)
group by a.custid;

##### 3a. Write a query to calculate how much payment is received from each customer in the 1st 6 months of them being a customer (across loans, if multiple loans are taken within their 1st 6 months).


select l.custid
	,sum(h.amount_paid)+sum(h.principal_paid) as total_paid
from all_loans l
join all_loanhist h
on l.loanid=h.loanid
where eowdate <= l.approvedate + INTERVAL '6 months'
group by l.custid;

##### 3b. Also provide what % of principal was collected in the 1st 6 months from the customer.

select custid, loanid,
	round((a.principal/a.total_loan)::numeric,2)*100 as perc_paid
from (
select l.custid, l.loanid,sum(h.principal_paid) as principal
, (select sum(l1.amount)
	   from all_loans l1 where l1.custid=l.custid
   		and l1.loanid=l.loanid
	   group by l1.custid) as total_loan
from all_loans l
join all_loanhist h
on l.loanid=h.loanid
where h.eowdate <= l.approvedate + INTERVAL '6 months'
group by l.custid,l.loanid) a
--group by custid, loanid
;

##### 4. Calculate the average rate of missing 1st payment by month of approvedate of loan.

select (a.pymnt_miss*100/a.total) as miss_rte_pct
from(
select count(l.loanid) as total
, (select count(l1.loanid) from all_loans l1
  where l1.loanid not in (select distinct loanid from all_loanhist
						 where eowdate <= l1.approvedate + INTERVAL '1 month')) as pymnt_miss
 from all_loans l
	) a;

###### 5. Calculate the top 3 most profitable customers in the tables. Profitability is defined as percentage of total paid of loan amount.

select a.custid, round((a.total_paid/a.total_loan)::numeric,2) as profitability
from(
select l.custid
	,sum(h.amount_paid) as total_paid
	, (select sum(l1.amount)
	   from all_loans l1 where l1.custid=l.custid
	   group by l1.custid) as total_loan
from all_loans l
join all_loanhist h
on l.loanid=h.loanid
group by l.custid)a
order by profitability desc
limit 3
;